In [1]:
import os
current_directory = os.getcwd()
folder_name = os.path.basename(current_directory)
number = folder_name

In [2]:
import platform
platform_name = platform.system()

In [3]:
CONFIG = {
    'test_path': 'test.csv',
    'train_path': 'train.csv',

    'DEVICE' : 'cuda',
    'SEED' : 42,
}
if platform_name == 'Windows':
    CONFIG['data_main'] = 'C:/Users/Николай/PycharmProjects/CIBMTR/D.Data/main/'
    CONFIG['model_path'] = f"{number}_model.pkl"
else:
    CONFIG['data_main'] = '/kaggle/input/equity-post-HCT-survival-predictions/'
    CONFIG['model_path'] = "/kaggle/input/2.1.14_model.pkl/other/default/1/2.1.14_model.pkl"

In [4]:

import pandas as pd
import joblib 

In [5]:
train = pd.read_csv(f"{CONFIG['data_main']}{CONFIG['train_path']}")
train.drop(columns=['ID'], inplace=True)

num_columns = ['y', 'efs', 'efs_time', 'age_at_hct', 'donor_age', 'ID']
cat_columns = [col for col in train.columns if col not in num_columns]

for col in train[['age_at_hct', 'donor_age']].columns:
    train[col] = train[col].fillna(train[col].mode()[0])
for col in train[cat_columns].columns:
    train[col] = train[col].astype(str)
    j_ch = ',[]{}:"\\<'  # набор символов для удаления
    for ch in j_ch:
        train[col] = train[col].apply(lambda x: str(x).replace(ch, ''))
train_one_hot = pd.get_dummies(train[cat_columns])
train_one_hot = pd.concat([train_one_hot, train[['age_at_hct', 'donor_age']]], axis=1)

In [6]:
test = pd.read_csv(f"{CONFIG['data_main']}test.csv")

for col in test[['age_at_hct', 'donor_age']].columns:
    test[col] = test[col].fillna(test[col].mode()[0])
for col in test[cat_columns].columns:
    test[col] = test[col].astype(str)
    j_ch = ',[]{}:"\\<'  # набор символов для удаления
    for ch in j_ch:
        test[col] = test[col].apply(lambda x: str(x).replace(ch, ''))
test_one_hot = pd.get_dummies(test[cat_columns])
test_one_hot = pd.concat([test_one_hot, test[['age_at_hct', 'donor_age']]], axis=1)

missing_columns = set(train_one_hot.columns) - set(test_one_hot.columns)
for col in missing_columns:
    test_one_hot[col] = False

C:\Users\Николай\AppData\Local\Temp\ipykernel_14644\2181879771.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_one_hot[col] = False
C:\Users\Николай\AppData\Local\Temp\ipykernel_14644\2181879771.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_one_hot[col] = False
C:\Users\Николай\AppData\Local\Temp\ipykernel_14644\2181879771.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at o

In [7]:
test_one_hot = test_one_hot[train_one_hot.columns]

In [8]:
final_cat = joblib.load(CONFIG['model_path'])

In [9]:
preds_test = final_cat.predict(test_one_hot)

In [10]:
preds_test

array([0.49605283, 0.60786125, 0.40154912])

In [11]:
y_pred = test[['ID']].copy(deep=True)
y_pred["prediction"] = preds_test
y_pred.to_csv('submission.csv', index=False)